<h1 style="text-align:center;">Add new user account by iLO privileges or redfish standard roles</h1>

<div class="alert  alert-block  alert-info" style="border-radius: 20px;">    
    <ul>
        <li><p>This Jupyter Notebook will help you to add new user account by iLO privileges or redfish standard roles. This code is written in Python and uses the redfish library to perform the required operations.</p></li>
        <li>As you are here let's get it done easily.</li>
        <li>To perform this u need to have <a href = "https://libraries.io/pypi/python-ilorest-library#id3" style="none">python-ilorest-library</a> installed in your machine.</li>
        <li>Please keep your username and password handy.</li>
        <li>Do execute each successive cell once to avoid any errors!</li>
    </ul>    
</div>

<div class="alert alert-block alert-success"  style="border-radius: 20px;">
    <h3> 1. Below we are importing all the necessary python libraries</h3>
</div>

In [ ]:
import sys
import json
from redfish import RedfishClient
from redfish.rest.v1 import ServerDownOrUnreachableError
global DISABLE_RESOURCE_DIR

from ilorest_util import get_resource_directory
from ilorest_util import get_gen

#to take password inputs securely
import getpass

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3> 2. This is our add_ilo_user_account function overview, in the end this will add new iLO user account. This function will be called only if the generation of iLO is not 4.</h3><br>
    
 There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to add new iLO user account.<br>

1. This function takes in six parameters: <b>_redfishobj, new_loginname, new_username, new_password, role_id, and privilege_dict.</b>
<br>
    
2. The function first calls the <b>get_resource_directory()</b> function by passing <b>_redfishobj</b> as the parameter to retrieve the resource directory of the server. If resource directory is not available or it is disabled, it finds the relevant URI by navigating through the server's API.
    <br>

3. If the resource directory is available, the function obtains all the account instances from the resource directory and sets the <b>account_collection_uri</b>. If it's an HPE server, it sets the <b>body</b> dictionary with the privileges passed in <b>privilege_dict</b> parameter.
    <br>

4. The function then fills the <b>body</b> dictionary with the necessary details like <b>new_username, new_password, and RoleId/Oem</b> depending on the type of server.
    <br>

5. The function then sends a POST command by passing the <b>account_collection_uri</b> and <b>body</b> dictionary as the parameters to create a new user account.
    <br>

6. If the iLO server responds with an HTTP status code outside of 200 or 201, the function prints an error message. If the HTTP response status code is 400, it prints the iLO extended info error message. If the HTTP response status code is 200 or 201, it prints a success message and the newly created user account details in JSON format.
    
    <br>

7. If bios_boot_uri is None, print an error message.
    

 </div>

In [ ]:
def add_ilo_user_account(_redfishobj, new_loginname, new_username, new_password, role_id, privilege_dict):
    resource_instances = get_resource_directory(_redfishobj)

    if DISABLE_RESOURCE_DIR or not resource_instances:
        #resource directory is not available so we will navigate through paths manually to obtain
        #account info
        account_service_uri = _redfishobj.root.obj['AccountService']['@odata.id']
        account_service_response = _redfishobj.get(account_service_uri)
        account_collection_uri = account_service_response.obj['Accounts']['@odata.id']
        #Add via role id
        body = {"RoleId": role_id}
    else:
        #obtain all account instances from resource directory
        for instance in resource_instances:
            if '#ManagerAccountCollection.' in instance['@odata.type']:
                account_collection_uri = instance['@odata.id']
        body = {"Oem": {"Hpe": {"Privileges": {}}}}
        #HPE server, so add via privileges
        for priv in privilege_dict:
            body["Oem"]["Hpe"]["Privileges"][priv] = privilege_dict[priv]
        #Add login name
        body["Oem"]["Hpe"]["LoginName"] = new_loginname

    #Fill in the rest of the payload
    body["UserName"] = new_username
    body["Password"] = new_password

    #We pass the URI and the dictionary as a POST command (part of the redfish object)
    resp = _redfishobj.post(account_collection_uri, body)

    #If iLO responds with soemthing outside of 200 or 201 then lets check the iLO extended info
    #error message to see what went wrong
    if resp.status == 400:
        try:
            print(json.dumps(resp.obj['error']['@Message.ExtendedInfo'], indent=4, sort_keys=True))
        except Exception:
            sys.stderr.write("A response error occurred, unable to access iLO Extended Message Info...")
    elif not resp.status in [200, 201]:
        sys.stderr.write("An http response of '%s' was returned.\n" % resp.status)
    else:
        print("Success!\n")
        print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
    <h3>This function adds a new iLO user account in the HPE Gen9 server or having iLO generation 4 servers.</h3><br>
    
    
1. The function takes <b>_redfishobj, new_loginname, new_username, and new_password</b> as input parameters..
    <br>
    
2. The account collection URI for the HPE Gen9 server is initialized as <b>/redfish/v1/AccountService/Accounts/</b>.
    <br>

3. The privilege dictionary for the HPE Gen9 server is constructed in the body variable. The privileges are specified for the iLO user account to be added. The login name is also added.
    <br>

4. The username and password for the iLO user account to be added are added to the <b>body</b> dictionary.
    <br>

5. The <b>POST</b> command is passed to <b>_redfishobj</b> to add the iLO user account using the <b>account_collection_uri</b> and <b>body</b> dictionary.
    <br>

6. If the HTTP response status code is 200 or 201, the newly created user account details are printed using json.dumps().
    
    <br>

7. If the HTTP response status code is not 200 or 201, an error message is printed using sys.stderr.write().
    

</div>

In [ ]:
def add_ilo_user_account_gen9(_redfishobj, new_loginname, new_username, new_password):

    account_collection_uri = "/redfish/v1/AccountService/Accounts/"
    #Add via gen9 priv dic
    body = {'Oem': {'Hp': {'Privileges': {"LoginPriv": True, "RemoteConsolePriv": True,
        "UserConfigPriv": True, "VirtualMediaPriv": True, "VirtualPowerAndResetPriv": True,
        "iLOConfigPriv": True}, 'LoginName': new_loginname}},'UserName': new_username, 'Password': new_password}
    #We pass the URI and the dictionary as a POST command (part of the redfish object)
    resp = _redfishobj.post(account_collection_uri, body)
    print(json.dumps(resp.dict, indent=4, sort_keys=True))

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
      <h3> 3. This is Main function overview.</h3>
      <br>
   There is a hierarchy involved in iLO Rest Api. So we are simply following the hierarchy present to add new user.
  

   1. The follwoing Main function prompts to take SYSTEM_URL, LOGIN_ACCOUNT, LOGIN_PASSWORD, ACCOUNT_LOGIN_NAME, ACCOUNT_USER_NAME, ACCOUNT_PASSWORD, ROLE_ID and PRIVILAGES as inputs.<br>
  <br>
    
    
  2. The rest opertions are handled by redfish object which is created by RedfishClient<br>
  <br>
      
  3. <b>RedfishClient</b> class takes 3 parameters<i> iLO url, Username,Password</i> and returns a redfish object. If users want to use create RedfishClient using "certificate" for authentication (which is optional), they can do so by uncommenting the lines ca_cert_data, ca_cert_data, ca_cert_data,ca_cert_data and providing their values.<br>
      <br>
  
  4. Once you enter everything a redfish obejct is created with the help of RedfishClient class.<br>
      <br>
  
  5. Now we try to Login with the help of <i> REDFISHOBJ</i>, if it is successful a session is created and you are logged in, But if           Server       is not avaialbe we get an error saying "server unreachable"<br>
      <br>
  6. Then we call <b>add_ilo_user_account</b> or <b>add_ilo_user_account_gen9</b> based on the generation of iLO by pssing appropriate as parameters to those functions.<br>
      
  
  7. REDFISHOBJ.logout() will take care of logging you out of the ilo.
  
   </div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
<b> When running on the server locally,</b> use the following values:<br>
    SYSTEM_URL = None <br>
    LOGIN_ACCOUNT = None <br>
    LOGIN_PASSWORD = None <br>
    <b> When running remotely </b> connect using the secured (https://) address, account name and password to send https requests<br>
    SYSTEM_URL acceptable examples: <br>
    "https://10.0.0.100" <br>
    "https://ilo.hostname"
</div>

<div class="alert alert-block alert-warning" style="border-radius: 20px;">
    <h5>Instructions</h5>
    <ul>
        <li>When prompted for <i>ACCOUNT_LOGIN_NAME</i>, enter redfish standard username here  as iLO GUI actually considers this to be 'UserName'</li>
        <li>When prompted for <i>ACCOUNT_USER_NAME</i>, enter redfish login here iLO GUI actually considers this to be 'LoginName'</li>
        <li>Role_ID: It is a predefined role for the user which can take "Administrator" or "ReadOnly" or "Operator" as input. For more informaion on roles and privilages refere the table in the next cell.</li>
        <li>For more clarification on the above points please visit <i><a href="https://hewlettpackard.github.io/ilo-rest-api-docs/ilo6/#managing-ilo-6-users">iLO Users Documentation</a></i></li>
    </ul>
</div>

<div class = "alert alert-block alert-success" style="border-radius: 20px;">
<h5>Roles and Privileges</h5>
    <p>iLO uses a set of privileges assigned to each user account to grant and restrict access to features. iLO 6’s privileges are:</p><br>
    
<table style="background-color: #f9f9f9;border-radius: 5px;">
  <thead>
    <tr>
      <th style="text-align: left;">Redfish</th>
      <th style="text-align: left;">iLO Web Interface (GUI)</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left;">LoginPriv</td>
      <td style="text-align: left;">Login</td>
    </tr>
    <tr>
      <td style="text-align: left;">RemoteConsolePriv</td>
      <td style="text-align: left;">Remote Console</td>
    </tr>
    <tr>
      <td style="text-align: left;">VirtualPowerAndResetPriv</td>
      <td style="text-align: left;">Virtual Power and Reset</td>
    </tr>
    <tr>
      <td style="text-align: left;">VirtualMediaPriv</td>
      <td style="text-align: left;">Virtual Media</td>
    </tr>
    <tr>
      <td style="text-align: left;">HostBIOSConfigPriv</td>
      <td style="text-align: left;">Host BIOS</td>
    </tr>
    <tr>
      <td style="text-align: left;">iLOConfigPriv</td>
      <td style="text-align: left;">Configure iLO Settings</td>
    </tr>
    <tr>
      <td style="text-align: left;">UserConfigPriv</td>
      <td style="text-align: left;">Administer User Accounts</td>
    </tr>
    <tr>
      <td style="text-align: left;">HostNICConfigPriv</td>
      <td style="text-align: left;">Host NIC</td>
    </tr>
    <tr>
      <td style="text-align: left;">HostStorageConfigPriv</td>
      <td style="text-align: left;">Host Storage</td>
    </tr>
    <tr>
      <td style="text-align: left;">SystemRecoveryConfigPriv</td>
      <td style="text-align: left;">Recovery Set</td>
    </tr>
  </tbody>
</table>

<h5>Roles</h5>
<p>You can use RoleId to create users with specific starting privileges.<br>
On a GET of the local user account, RoleId is synthesized based upon the enabled privileges. iLO 6 does not store a separate RoleId value. For this reason, modifications to raw privileges may or may not result in a changed RoleId based upon iLO’s mapping.<br>            
If the PATCH includes both RoleID and individual privileges, the privileges corresponding to the RoleId are assigned to the local user account first, and then the explicit privileges are assigned.</p>

<h5>Privileges granted on Local Account Creation by RoleId</h5>
<br>
<table style="background-color: #f9f9f9;border-radius: 5px;">
  <thead>
    <tr>
      <th style="text-align: left;">RoleId</th>
      <th style="text-align: left;">Privileges</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left;">Administrator</td>
      <td style="text-align: left;">HostBIOSConfigPriv, HostNICConfigPriv, HostStorageConfigPriv, LoginPriv, RemoteConsolePriv, UserConfigPriv, VirtualMediaPriv, VirtualPowerAndResetPriv, iLOConfigPriv</td>
    </tr>
    <tr>
      <td style="text-align: left;">Operator</td>
      <td style="text-align: left;">HostBIOSConfigPriv, HostNICConfigPriv, HostStorageConfigPriv, LoginPriv, RemoteConsolePriv, VirtualMediaPriv, VirtualPowerAndResetPriv</td>
    </tr>
    <tr>
      <td style="text-align: left;">ReadOnly</td>
      <td style="text-align: left;">LoginPriv</td>
    </tr>
  </tbody>
</table>


<h5>RoleId shown on an existing Local User Account by Privilege</h5>
<p>The RoleId reported is the smallest superset of assigned privileges.</p>
<br>
<table style="background-color: #f9f9f9;border-radius: 5px;">
  <thead>
    <tr>
      <th style="text-align: left;">Privileges</th>
      <th style="text-align: left;">RoleId</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="text-align: left;">LoginPriv only</td>
      <td style="text-align: left;">ReadOnly</td>
    </tr>
    <tr>
      <td style="text-align: left;">iLOConfigPriv or UserConfigPriv or SystemRecoveryConfigPriv and anything else</td>
      <td style="text-align: left;">Administrator</td>
    </tr>
    <tr>
      <td style="text-align: left;">any other combination</td>
      <td style="text-align: left;">Operator</td>
    </tr>
  </tbody>
</table>

  

</div>

In [ ]:
if __name__ == "__main__":

    
    SYSTEM_URL = input("Enter System URL: ")
    LOGIN_ACCOUNT = input("Enter Login account: ")
    LOGIN_PASSWORD = getpass.getpass("Enter your password: ")

    #account login name (iLO GUI actually considers this to be 'UserName', but
    #this is the redfish standard username)
    ACCOUNT_LOGIN_NAME = input("Enter Account Login Name: ")

    #account user name (iLO GUI actually considers this to be 'LoginName', but
    #this is the redfish login)
    ACCOUNT_USER_NAME = input("Enter Account User Name: ")

    #account password
    ACCOUNT_PASSWORD = getpass.getpass("Enter Account password: ")

    #A predefined role for the user, (The redfish standard method for accounts).
    #This is a translated to a pre-configured arrangement of privileges on HPE servers
    ROLE_ID = input("Enter ROLE_ID: ") #Administrator, ReadOnly or Operator are available

    #Dictionary of modifiable privileges for HPE servers (modify this if you wish to directly set
    #an account with specific privileges
    
    PRIVILEGE_DICT = {"iLOConfigPriv": True, "VirtualMediaPriv": True, "RemoteConsolePriv": True, "UserConfigPriv": True, "VirtualPowerAndResetPriv": True, \
                      "SystemRecoveryConfigPriv": True, "LoginPriv": True, "HostStorageConfigPriv": True, "HostNICConfigPriv": True,
                      "HostBIOSConfigPriv": True}
    
    print("For the below privilages enter 'True' if that privilages needs to be set for the user else enter 'False'")
          
    for key in PRIVILEGE_DICT.keys():
          PRIVILEGE_DICT[key] = input(f"{key}")
    
    # flag to force disable resource directory. Resource directory and associated operations are
    # intended for HPE servers.
    DISABLE_RESOURCE_DIR = False

#     ca_cert_data = {}
#     ca_cert_data["cert_file"] = "c:\\test\\ppcacuser.crt"
#     ca_cert_data["key_file"] = "c:\\test\\ppcacuserpriv.key"
#     ca_cert_data["key_password"] = "password"
    LOGIN_ACCOUNT = None
    LOGIN_PASSWORD = None

    try:
        # Create a Redfish client object
        REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, username=LOGIN_ACCOUNT, password=LOGIN_PASSWORD, ca_cert_data=ca_cert_data)
        #REDFISHOBJ = RedfishClient(base_url=SYSTEM_URL, ca_cert_data=ca_cert_data)
        # Login with the Redfish client
        if ca_cert_data is None:
            REDFISHOBJ.login()
        else:
            REDFISHOBJ.login(auth='certificate')
    except ServerDownOrUnreachableError as excp:
        sys.stderr.write("ERROR: server not reachable or does not support RedFish.\n")
        sys.exit()


    #if account_collection_uri and accounts:
    #add specified account
    (ilogen,_) = get_gen(REDFISHOBJ)
    print ("Generation is ", ilogen)
    if int(ilogen) !=4:
        add_ilo_user_account(REDFISHOBJ,ACCOUNT_LOGIN_NAME,ACCOUNT_USER_NAME,ACCOUNT_PASSWORD,ROLE_ID,PRIVILEGE_DICT)
    else:
        add_ilo_user_account_gen9(REDFISHOBJ,ACCOUNT_LOGIN_NAME,ACCOUNT_USER_NAME,ACCOUNT_PASSWORD)
        
    REDFISHOBJ.logout()

Enter System URL: f
Enter Login account: f
Enter your password: ········
Enter Account Login Name: f
Enter Account User Name: f
Enter Account password: ········
Enter ROLE_ID: f
For the below privilages enter 'True' if that privilages needs to be set for the user else enter 'False'
Truef
Truef
Truef
Truef
